In [50]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from datetime import time
import os
import math



In [51]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 0)

In [52]:
raw_file="Raw data 29-April'25 According to 4_7.5.csv"
gst_file="GST 29-April'25 According to 4_7.5.csv"
tracker_file="EDD Tracker till 29th April'25.csv"
format_file="edd_format.csv"
ops_file="ops_status.csv"
old_tracker_file="old_tracker.csv"


In [53]:
raw_data= pd.read_csv(raw_file,memory_map=True)
gst_data=pd.read_csv(gst_file,memory_map=True)
tracker_data=pd.read_csv(tracker_file,memory_map=True)
master_format=pd.read_csv(format_file,memory_map=True)
ops_status=pd.read_csv(ops_file,memory_map=True)
old_tracker=pd.read_csv(old_tracker_file,memory_map=True)

In [54]:
raw_data.head(3)

,parent_entity_id,parent_id,mid,entity_id,mid_creation_date,channel_date,cust_id,channel,edc_flag,vpa_type,sellerid,bank_response,bank_name,pan_no_business,pan_no_personal,gstin,gst_flag,category,sub_category,upi_mcc_code,entity_type,gmv,limit_flag,drr,month_expected_gmv,total_child_mid,rank,limit_usage,_required_flag,pending_at_,propose_limit
0,84453060,nOMFCR01645828915430,nOMFCR01645828915430,84453060,2024-07-17 14:01:19.000,2024-07-17 14:01:47.000,3.057817e+08,DIY,QR,PTY,07084453060,"{""sellerId"":""07084453060""}",PTYES,ABSFA8213A,ABSFA8213A,NaN,No,Healthcare,Pathology Lab,8071,PARTNERSHIP,157000.0,400000,7476.190476,231761.904762,1,1,0.392500,Not_required,Pending for share,300000.0
1,84347054,nRDByj92395804370708,nRDByj92395804370708,84347054,2024-07-05 14:26:24.000,2024-07-05 14:26:51.000,1.211667e+09,Petrol Pumps,QR,PTY,10884347054,"{""sellerId"":""10884347054""}",PTYES,CMYPK5082N,CMYPK5082N,02CMYPK5082N1ZB,Yes,Beauty and Wellness,Salon and Parlour,7230,PROPRIETORSHIP,30635.0,750000,1225.400000,37987.400000,1,1,0.040847,Not_required,Pending for share,100000.0
2,84001708,nSqGBT98393941957833,nSqGBT98393941957833,84001708,2024-05-22 17:16:15.000,2024-05-22 17:16:40.000,1.894605e+09,Offline Unorganized,QR,PTY,15484001708,"{""sellerId"":""15484001708""}",PTYES,OJBPK0545J,OJBPK0545J,09OJBPK0545J1ZW,Yes,Retail and Shopping,Grocery,5411,PROPRIETORSHIP,NaN,750000,NaN,NaN,1,1,NaN,Not_required,Pending for share,NaN


In [55]:
gst_data=gst_data.rename(columns={'_col3':'updated_at','solution_value.1':'gst'})


# Convert 'date' column to datetime type
gst_data['updated_at'] = pd.to_datetime(gst_data['updated_at'])


gst_data.head(3)

,solution_value,cust_id,gst,updated_at
0,qvykNp56833717299221,1.444497e+09,Slab: Rs. 0 to 40 lakhs,2025-04-29
1,HsvYKV54344606546086,1.496214e+09,Slab: Rs. 1.5 Cr. to 5 Cr.,2025-04-29
2,HsvYKV54344606546086,1.496214e+09,Slab: Rs. 1.5 Cr. to 5 Cr.,2025-04-29


In [56]:


# Sort by id and date (latest date first)
gst_data = gst_data.sort_values(by=['solution_value', 'updated_at'], ascending=[True, False])

# Drop duplicates: keep the first (which is the latest date because of sort)
gst_data = gst_data.drop_duplicates(subset='solution_value', keep='first')

# Reset index if needed
gst_data = gst_data.reset_index(drop=True)

In [57]:
# Convert the two columns to datetime first
raw_data['mid_creation_date'] = pd.to_datetime(raw_data['mid_creation_date'], errors='coerce')
raw_data['channel_date'] = pd.to_datetime(raw_data['channel_date'], errors='coerce')

# Then format them to 'dd-mm-yyyy' string format
raw_data['mid_creation_date'] = raw_data['mid_creation_date'].dt.strftime('%d-%m-%Y')
raw_data['channel_date'] = raw_data['channel_date'].dt.strftime('%d-%m-%Y')

raw_data.head(3)

,parent_entity_id,parent_id,mid,entity_id,mid_creation_date,channel_date,cust_id,channel,edc_flag,vpa_type,sellerid,bank_response,bank_name,pan_no_business,pan_no_personal,gstin,gst_flag,category,sub_category,upi_mcc_code,entity_type,gmv,limit_flag,drr,month_expected_gmv,total_child_mid,rank,limit_usage,_required_flag,pending_at_,propose_limit
0,84453060,nOMFCR01645828915430,nOMFCR01645828915430,84453060,17-07-2024,17-07-2024,3.057817e+08,DIY,QR,PTY,07084453060,"{""sellerId"":""07084453060""}",PTYES,ABSFA8213A,ABSFA8213A,NaN,No,Healthcare,Pathology Lab,8071,PARTNERSHIP,157000.0,400000,7476.190476,231761.904762,1,1,0.392500,Not_required,Pending for share,300000.0
1,84347054,nRDByj92395804370708,nRDByj92395804370708,84347054,05-07-2024,05-07-2024,1.211667e+09,Petrol Pumps,QR,PTY,10884347054,"{""sellerId"":""10884347054""}",PTYES,CMYPK5082N,CMYPK5082N,02CMYPK5082N1ZB,Yes,Beauty and Wellness,Salon and Parlour,7230,PROPRIETORSHIP,30635.0,750000,1225.400000,37987.400000,1,1,0.040847,Not_required,Pending for share,100000.0
2,84001708,nSqGBT98393941957833,nSqGBT98393941957833,84001708,22-05-2024,22-05-2024,1.894605e+09,Offline Unorganized,QR,PTY,15484001708,"{""sellerId"":""15484001708""}",PTYES,OJBPK0545J,OJBPK0545J,09OJBPK0545J1ZW,Yes,Retail and Shopping,Grocery,5411,PROPRIETORSHIP,NaN,750000,NaN,NaN,1,1,NaN,Not_required,Pending for share,NaN


In [58]:
tracker_data=tracker_data.fillna(0)
tracker_data.isna().sum()

Bank Response    0
Final Remarks    0
dtype: int64

In [59]:
len(ops_status)+len(tracker_data)+len(old_tracker)

319432

In [60]:
tracker_data=tracker_data.rename(columns={'Bank Response':'bank_response','Final Remarks':'final_remarks'})
old_tracker=old_tracker.rename(columns={'bank_response':'bank_response','Final Remarks':'final_remarks'})
ops_status=ops_status.rename(columns={'sellerid':'bank_response','OPS status':'final_remarks'})



ops_status_data= pd.concat([ops_status, tracker_data,old_tracker], axis=0, ignore_index=True)
print('totol tracker data ',len(ops_status_data))
print(ops_status_data.isna().sum())


totol tracker data  319432
bank_response    0
final_remarks    0
dtype: int64


In [61]:
len(raw_data)

321004

In [62]:
# Make sure everything is a string first
raw_data['sellerid'] = raw_data['sellerid'].astype(str)
raw_data['entity_id'] = raw_data['entity_id'].astype(str)
raw_data['parent_entity_id'] = raw_data['parent_entity_id'].astype(str)

# Calculate the length of each entity_id
raw_data['entity_len'] = raw_data['entity_id'].str.len()

# Extract from seller_id the last N characters based on entity_len
raw_data['seller_tail'] = raw_data.apply(lambda x: x['sellerid'][-int(x['entity_len']):], axis=1)


# Step 3: Compare both
raw_data['is_entity_match'] = raw_data['seller_tail'] == raw_data['entity_id']
raw_data['is_parent_match'] = raw_data['seller_tail'] == raw_data['parent_entity_id']

# Step 4: Create final match column
raw_data['final_match'] = raw_data['is_entity_match'] | raw_data['is_parent_match']

raw_data = raw_data[raw_data['final_match']]


len(raw_data)



280033

In [63]:
master_format[['parent_id', 'Child MID', 'Parent Entity_Id', 'Child Entity_Id',
       'MID Creation Date', 'PTY Creation Date', 'Cust_id', 'channel',
       'EDC Flag', 'vpa_type', 'sellerid', 'bank_response', 'bank_name',
       'Pan No Business', 'Pan No Personal', 'gstin', 'gst_flag', 'category',
       'sub_category', 'upi_mcc_code', 'entity_type', 'gmv', 'limit_flag',
       'DRR', 'Month_expected_GMV', 'total_Child_mid', 'rank']]=raw_data[['parent_id', 'mid','parent_entity_id','entity_id',
       'mid_creation_date', 'channel_date', 'cust_id', 'channel', 'edc_flag',
       'vpa_type', 'sellerid', 'bank_response', 'bank_name', 'pan_no_business',
       'pan_no_personal', 'gstin', 'gst_flag', 'category', 'sub_category',
       'upi_mcc_code', 'entity_type', 'gmv', 'limit_flag', 'drr',
       'month_expected_gmv', 'total_child_mid', 'rank']]

len(master_format)

280033

In [64]:
gst_mapping = gst_data.set_index('solution_value')['gst'].to_dict()
master_format['GST Turn over'] = master_format['parent_id'].map(gst_mapping)

# Fill missing GSTs with 'NA'
master_format['GST Turn over'] = master_format['GST Turn over'].fillna('NA')

In [65]:
ops_status_mapping = ops_status_data.set_index('bank_response')['final_remarks'].to_dict()
master_format['OPS team status'] = master_format['bank_response'].map(ops_status_mapping)

# Fill missing GSTs with 'NA'
master_format['OPS team status'] = master_format['OPS team status'].fillna('Pending for share')

In [66]:
master_format.columns

Index(['parent_id', 'Child MID', 'Parent Entity_Id', 'Child Entity_Id',
       'MID Creation Date', 'PTY Creation Date', 'Cust_id', 'channel',
       'EDC Flag', 'vpa_type', 'sellerid', 'bank_response', 'bank_name',
       'Pan No Business', 'Pan No Personal', 'gstin', 'gst_flag', 'category',
       'sub_category', 'upi_mcc_code', 'entity_type', 'gmv', 'limit_flag',
       'DRR', 'Month_expected_GMV', 'total_Child_mid', 'rank',
       'Proposed Limit', '% usage', 'Grouping', 'GST Turn over',
       'Liquor/Others', 'Limit Required/Not', 'Business status',
       'OPS team status', 'YBL status', 'Sum of GMV', 'Sum of DRR',
       'Sum of Month Expected GMV', 'Sum of limit_flag',
       'Sum of Proposed Limit', 'Limit Required/Not.1'],
      dtype='object')

In [67]:

master_format['Proposed Limit'] = np.where(
    master_format['Month_expected_GMV'].isna(), 
    master_format['limit_flag'], 
    np.maximum(np.ceil(master_format['Month_expected_GMV'] / 1e5) * 1e5, master_format['limit_flag'])
)


c:\ProgramData\Anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: invalid value encountered in ceil
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [68]:

# First replace NaN in 'gmv' with 0 for calculation
master_format['% usage'] = (master_format['gmv'].fillna(0) / master_format['limit_flag']) * 100




In [69]:

# Make sure % usage is numeric
master_format['% usage'] = pd.to_numeric(master_format['% usage'], errors='coerce')

# Define bins and labels
bins = [-float('inf'), 5, 15, 40, 60, 80, 100, float('inf')]
labels = ['1_<5 %', '2_5 to 15 %', '3_15-40%', '4_40-60%', '5_60-80%', '6_80-100%', '7.above 100%']

# Apply binning
master_format['Grouping'] = pd.cut(master_format['% usage'], bins=bins, labels=labels, right=False)



master_format['Grouping'].value_counts()


1_<5 %          147916
7.above 100%     48525
3_15-40%         31039
2_5 to 15 %      27484
4_40-60%         12006
5_60-80%          8121
6_80-100%         4942
Name: Grouping, dtype: int64

In [70]:

# Suppose your DataFrame is named df
liquor_categories = ['Alcoholic Beverages', 'Restaurants and Bars', 'wine and liquor shop', 'Wine producers','Alcoholic beverages']

master_format['Liquor/Others'] = master_format['sub_category'].apply(lambda x: 'Liquor' if x in liquor_categories else 'Others')


master_format['Liquor/Others'].value_counts()



Others    267725
Liquor     12308
Name: Liquor/Others, dtype: int64

In [71]:


master_format['Limit Required/Not'] = np.where(master_format['Proposed Limit'] > master_format['limit_flag'], 'Required', 'Not_required')

master_format['Limit Required/Not'].value_counts()


Not_required    220576
Required         59457
Name: Limit Required/Not, dtype: int64

In [72]:

master_format['Sum of GMV'] = master_format.groupby('bank_response')['gmv'].transform('sum')

master_format['Sum of DRR'] = master_format.groupby('bank_response')['DRR'].transform('sum')

master_format['Sum of Month Expected GMV'] = master_format.groupby('bank_response')['Month_expected_GMV'].transform('sum')

master_format['Sum of limit_flag'] = master_format.groupby('bank_response')['limit_flag'].transform('sum')

# Use numpy's vectorized operations  (MAX and ROUNDUP equivalent)
master_format['Sum of Proposed Limit'] = np.maximum(np.ceil(master_format['Sum of Month Expected GMV'] / 10**5) * 10**5, master_format['Sum of limit_flag'])

# Use numpy for the IF condition (AO2 > AN2)
master_format['Limit Required/Not.1'] = np.where(master_format['Sum of Proposed Limit'] > master_format['Sum of limit_flag'], "Required", "Not_required")

In [73]:
master_format.head(5)

,parent_id,Child MID,Parent Entity_Id,Child Entity_Id,MID Creation Date,PTY Creation Date,Cust_id,channel,EDC Flag,vpa_type,sellerid,bank_response,bank_name,Pan No Business,Pan No Personal,gstin,gst_flag,category,sub_category,upi_mcc_code,entity_type,gmv,limit_flag,DRR,Month_expected_GMV,total_Child_mid,rank,Proposed Limit,% usage,Grouping,GST Turn over,Liquor/Others,Limit Required/Not,Business status,OPS team status,YBL status,Sum of GMV,Sum of DRR,Sum of Month Expected GMV,Sum of limit_flag,Sum of Proposed Limit,Limit Required/Not.1
0,nOMFCR01645828915430,nOMFCR01645828915430,84453060,84453060,17-07-2024,17-07-2024,3.057817e+08,DIY,QR,PTY,07084453060,"{""sellerId"":""07084453060""}",PTYES,ABSFA8213A,ABSFA8213A,NaN,No,Healthcare,Pathology Lab,8071,PARTNERSHIP,157000.00,400000,7476.190476,231761.904762,1,1,400000.0,39.250000,3_15-40%,NA,Others,Not_required,NaN,Closed,NaN,157000.00,7476.190476,231761.904762,400000,400000.0,Not_required
1,nRDByj92395804370708,nRDByj92395804370708,84347054,84347054,05-07-2024,05-07-2024,1.211667e+09,Petrol Pumps,QR,PTY,10884347054,"{""sellerId"":""10884347054""}",PTYES,CMYPK5082N,CMYPK5082N,02CMYPK5082N1ZB,Yes,Beauty and Wellness,Salon and Parlour,7230,PROPRIETORSHIP,30635.00,750000,1225.400000,37987.400000,1,1,750000.0,4.084667,1_<5 %,Slab: Rs. 0 to 40 lakhs,Others,Not_required,NaN,Closed,NaN,30635.00,1225.400000,37987.400000,750000,750000.0,Not_required
2,nSqGBT98393941957833,nSqGBT98393941957833,84001708,84001708,22-05-2024,22-05-2024,1.894605e+09,Offline Unorganized,QR,PTY,15484001708,"{""sellerId"":""15484001708""}",PTYES,OJBPK0545J,OJBPK0545J,09OJBPK0545J1ZW,Yes,Retail and Shopping,Grocery,5411,PROPRIETORSHIP,NaN,750000,NaN,NaN,1,1,750000.0,0.000000,1_<5 %,NA,Others,Not_required,NaN,Pending for share,NaN,0.00,0.000000,0.000000,750000,750000.0,Not_required
4,zHkWwA99081997626879,zHkWwA99081997626879,84937045,84937045,09-09-2024,09-09-2024,4.449731e+06,Offline Unorganized,EDC,PTY,13284937045,"{""sellerId"":""13284937045""}",PTYES,BKHPB0327D,BKHPB0327D,21BKHPB0327D1Z5,Yes,CDIT,Electronics and Home Appliances,5722,PROPRIETORSHIP,6450.00,750000,1290.000000,39990.000000,1,1,750000.0,0.860000,1_<5 %,Slab: Rs. 0 to 40 lakhs,Others,Not_required,NaN,Uploaded,NaN,6450.00,1290.000000,39990.000000,750000,750000.0,Not_required
5,zLmYCa24008459623761,zLmYCa24008459623761,44993561,44993561,22-07-2019,22-04-2025,3.868290e+08,Offline Unorganized,EDC,PTY,01644993561,"{""sellerId"":""01644993561""}",PTYES,CREPK2019L,CREPK2019L,NaN,No,Retail and Shopping,Apparel,5137,PROPRIETORSHIP,17278.83,400000,3455.766000,107128.746000,1,1,400000.0,4.319707,1_<5 %,NA,Others,Not_required,NaN,Pending at YBL,NaN,17278.83,3455.766000,107128.746000,400000,400000.0,Not_required


In [ ]:
#master_format.to_csv('master_format.csv',index=False)

In [75]:
pending_data =master_format[master_format['OPS team status']=='Pending for share']
len(pending_data)

46653

In [76]:
pending_data['category'].unique()

array(['Retail and Shopping', 'Entertainment', 'Food', 'Agriculture',
       'Beauty and Wellness', 'Automobiles and Vehicles',
       'Individual Services', 'Healthcare', 'Travel', 'Business Services',
       'Grocery', 'Wholesale Trade and B2B', 'CDIT',
       'Cable and Broadband', 'Personal Services',
       'Milk Dairy and Cooperative', 'Event Services', 'GROCERY',
       'Corporate Alliance', 'LFR and Brands', 'Logistics',
       'Home Services', 'Education', 'Utilities', 'Devotion', 'NGO',
       'Tuition and Classes', 'Housing', 'Gas and Petrol', 'Airport',
       'Digital Goods', 'Hotel', 'Transport', 'Tolls',
       'Parking and Tolls', 'Government', 'Parking', 'BFSI'], dtype=object)

In [77]:
# Create a new column 'remark'
pending_data['Remark'] = ''

# Set remark as 'share' where cat is 'Gas and Petrol' or 'Gas Petrol'
pending_data.loc[pending_data['category'].isin(['Gas and Petrol', 'Gas Petrol']), 'Remark'] = 'O&G'

C:\Users\avnish1.yadav\AppData\Local\Temp\ipykernel_26832\2633286974.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pending_data['Remark'] = ''


In [78]:

special_cat = {
    'Category': [
        'Retail and Shopping', 'Food', 'Retail and Shopping', 'Wholesale Trade and B2B',
        'Healthcare', 'Healthcare', 'Education', 'Education', 'Education', 'Education',
        'Healthcare', 'Food','Retail and Shopping'
    ],
    'Subcategory': [
        'wine and liquor shop', 'Restaurants and Bars', 'Alcoholic Beverages', 'Wine producers',
        'Hospital', 'Pharmacy', 'School - Secondary & Higher Secondary', 'School - Primary',
        'School', 'Private College', 'Medical College', 'Food','Alcoholic beverages'
    ],
    'Main Group': [
        'Liquor', 'Liquor', 'Liquor', 'Liquor',
        'Hospital', 'Pharmacy', 'Education', 'Education',
        'Education', 'Education', 'Education', 'Food','Liquor'
    ]
}

special_category = pd.DataFrame(special_cat)

In [79]:
pending_data.columns

Index(['parent_id', 'Child MID', 'Parent Entity_Id', 'Child Entity_Id',
       'MID Creation Date', 'PTY Creation Date', 'Cust_id', 'channel',
       'EDC Flag', 'vpa_type', 'sellerid', 'bank_response', 'bank_name',
       'Pan No Business', 'Pan No Personal', 'gstin', 'gst_flag', 'category',
       'sub_category', 'upi_mcc_code', 'entity_type', 'gmv', 'limit_flag',
       'DRR', 'Month_expected_GMV', 'total_Child_mid', 'rank',
       'Proposed Limit', '% usage', 'Grouping', 'GST Turn over',
       'Liquor/Others', 'Limit Required/Not', 'Business status',
       'OPS team status', 'YBL status', 'Sum of GMV', 'Sum of DRR',
       'Sum of Month Expected GMV', 'Sum of limit_flag',
       'Sum of Proposed Limit', 'Limit Required/Not.1', 'Remark'],
      dtype='object')

In [80]:

# Perform a left merge to get the corresponding Main Group based on cat and sub_cat
pending_data = pd.merge(pending_data, special_category, how='left', left_on=['category', 'sub_category'], right_on=['Category', 'Subcategory'])

# Update the 'remark' column only where there is a match


pending_data['Remark'] = pending_data.apply(
    lambda row: row['Main Group'] if pd.notna(row['Main Group']) else row['Remark'],
    axis=1
)


pending_data = pending_data.drop(['Category', 'Subcategory','Main Group'], axis=1)

In [81]:
pending_data['Remark'].value_counts()

             46538
Liquor          53
Pharmacy        39
O&G             17
Education        4
Hospital         2
Name: Remark, dtype: int64

In [82]:
pending_data['bank_response_count'] = pending_data['bank_response'].map(pending_data['bank_response'].value_counts())

In [83]:
turn_over = {
    'GST Return Slab': [
        'without GSTIN only pan With Business Proof',
        'GSTIN Turnover NA',
        'Slab: Rs. 0 to 40 lakhs',
        'Slab: Rs. 40 lakhs to 1.5 Cr.',
        'Slab: Rs. 1.5 Cr. to 5 Cr.',
        'Slab: Rs. 5 Cr. to 25 Cr.',
        'Slab: Rs. 25 Cr. to 100 Cr.',
        'Slab: Rs. 100 Cr. to 500 Cr.',
        'Slab: Rs. 500 Cr. and above',
        'Slab: Rs. 25 Cr. to 100 Cr.',
        'Slab: Rs.0 to 40 lakhs',
        'Slab: Rs.1.5 Cr. to 5 Cr.',
        'Slab: Rs.25 Cr. to 100 Cr.',
        'Slab: Rs.40 lakhs to 1.5 Cr.',
        'Slab: Rs.5 Cr. to 25 Cr.',
        'NA'
    ],
    'Limit as per Slab': [
        600000,
        750000,
        750000,
        750000,
        2500000,
        6000000,
        10000000,
        10000000,
        10000000,
        10000000,
        750000,
        2500000,
        10000000,
        750000,
        6000000,
        600000
    ]
}

gst_turn_over = pd.DataFrame(turn_over)

In [84]:
# Remove duplicates for clean mapping
gst_turn_over = gst_turn_over.drop_duplicates('GST Return Slab')

# Create mapping dictionary
slab_to_limit = dict(zip(gst_turn_over['GST Return Slab'], gst_turn_over['Limit as per Slab']))


# Map the limits based on slab
pending_data['Limit as per Slab'] = pending_data['GST Turn over'].map(slab_to_limit)


In [85]:
pending_data.tail(3)

,parent_id,Child MID,Parent Entity_Id,Child Entity_Id,MID Creation Date,PTY Creation Date,Cust_id,channel,EDC Flag,vpa_type,sellerid,bank_response,bank_name,Pan No Business,Pan No Personal,gstin,gst_flag,category,sub_category,upi_mcc_code,entity_type,gmv,limit_flag,DRR,Month_expected_GMV,total_Child_mid,rank,Proposed Limit,% usage,Grouping,GST Turn over,Liquor/Others,Limit Required/Not,Business status,OPS team status,YBL status,Sum of GMV,Sum of DRR,Sum of Month Expected GMV,Sum of limit_flag,Sum of Proposed Limit,Limit Required/Not.1,Remark,bank_response_count,Limit as per Slab
46650,SMrIJb55167746761010,SMrIJb55167746761010,85423602,85423602,19-10-2024,19-10-2024,1.927838e+09,Petrol Pumps,EDC,PTY,03185423602,"{""sellerId"":""03185423602""}",PTYES,CMOPJ5742E,CMOPJ5742E,33CMOPJ5742E1Z1,Yes,Retail and Shopping,Mens and womens clothing shops,5691,PROPRIETORSHIP,86044.0,750000,3585.166667,111140.166667,1,1,750000.0,11.472533,2_5 to 15 %,NA,Others,Not_required,NaN,Pending for share,NaN,86044.0,3585.166667,111140.166667,750000,750000.0,Not_required,,1,600000
46651,SUPRAJ39877070539790,SUPRAJ39877070539790,83846898,83846898,07-03-2024,02-08-2024,3.393569e+08,Petrol Pumps,EDC,PTY,12183846898,"{""sellerId"":""12183846898""}",PTYES,ABSPL0212A,ABSPL0212A,27ABSPL0212A1Z6,Yes,Food,Restaurant,5812,PROPRIETORSHIP,NaN,750000,NaN,NaN,1,1,750000.0,0.000000,1_<5 %,Slab: Rs. 40 lakhs to 1.5 Cr.,Others,Not_required,NaN,Pending for share,NaN,0.0,0.000000,0.000000,750000,750000.0,Not_required,,1,750000
46652,SHREET97858478512936,SHREET97858478512936,84304675,84304675,29-06-2024,01-07-2024,1.341008e+09,Petrol Pumps,EDC,PTY,12384304675,"{""sellerId"":""12384304675""}",PTYES,CPZPG4237D,CPZPG4237D,29CPZPG4237D1ZI,Yes,Retail and Shopping,Mobile and Tablets,5732,PROPRIETORSHIP,NaN,750000,NaN,NaN,1,1,750000.0,0.000000,1_<5 %,NA,Others,Not_required,NaN,Pending for share,NaN,0.0,0.000000,0.000000,750000,750000.0,Not_required,,1,600000


In [86]:
pending_data.columns

Index(['parent_id', 'Child MID', 'Parent Entity_Id', 'Child Entity_Id',
       'MID Creation Date', 'PTY Creation Date', 'Cust_id', 'channel',
       'EDC Flag', 'vpa_type', 'sellerid', 'bank_response', 'bank_name',
       'Pan No Business', 'Pan No Personal', 'gstin', 'gst_flag', 'category',
       'sub_category', 'upi_mcc_code', 'entity_type', 'gmv', 'limit_flag',
       'DRR', 'Month_expected_GMV', 'total_Child_mid', 'rank',
       'Proposed Limit', '% usage', 'Grouping', 'GST Turn over',
       'Liquor/Others', 'Limit Required/Not', 'Business status',
       'OPS team status', 'YBL status', 'Sum of GMV', 'Sum of DRR',
       'Sum of Month Expected GMV', 'Sum of limit_flag',
       'Sum of Proposed Limit', 'Limit Required/Not.1', 'Remark',
       'bank_response_count', 'Limit as per Slab'],
      dtype='object')

In [87]:
pending_data['Remark'].value_counts()

             46538
Liquor          53
Pharmacy        39
O&G             17
Education        4
Hospital         2
Name: Remark, dtype: int64

In [88]:
# Apply the condition and update remark
condition = (
    (pending_data['Remark'].isna() | (pending_data['Remark'].str.strip() == '')) &
    (pending_data['gst_flag'].str.lower() == 'yes') &
    (pending_data['Sum of Proposed Limit'] > 750000)
)

pending_data.loc[condition, 'Remark'] = 'GST = YES & Sum of Proposed Limit > 750000'


In [89]:
pending_data['Remark'].value_counts()

                                              46499
Liquor                                           53
GST = YES & Sum of Proposed Limit > 750000       39
Pharmacy                                         39
O&G                                              17
Education                                         4
Hospital                                          2
Name: Remark, dtype: int64

In [90]:
# Apply the condition and update remark
condition = (
    (pending_data['Remark'].isna() | (pending_data['Remark'].str.strip() == '')) &
    (pending_data['gst_flag'].str.lower() == 'no') &
    (pending_data['Sum of Proposed Limit'] > 400000)
)

pending_data.loc[condition, 'Remark'] = 'GST = NO & Sum of Proposed Limit > 400000'

In [91]:
pending_data['Remark'].value_counts()

                                              46477
Liquor                                           53
GST = YES & Sum of Proposed Limit > 750000       39
Pharmacy                                         39
GST = NO & Sum of Proposed Limit > 400000        22
O&G                                              17
Education                                         4
Hospital                                          2
Name: Remark, dtype: int64

In [92]:
# Apply the condition and update remark
condition = (
    (pending_data['Remark'].isna() | (pending_data['Remark'].str.strip() == '')) &
    (pending_data['gst_flag'].str.lower() == 'yes') &
    ((pending_data['Sum of Proposed Limit'] - pending_data['Limit as per Slab'])< 0)
)

pending_data.loc[condition, 'Remark'] = 'GST = YES & (Proposed Limit < GST Slab Limit)'

In [93]:
pending_data['Remark'].value_counts()

                                                 46417
GST = YES & (Proposed Limit < GST Slab Limit)       60
Liquor                                              53
GST = YES & Sum of Proposed Limit > 750000          39
Pharmacy                                            39
GST = NO & Sum of Proposed Limit > 400000           22
O&G                                                 17
Education                                            4
Hospital                                             2
Name: Remark, dtype: int64

In [ ]:
master_format.to_csv('master_format.csv',index=False)

pending_data.to_csv('pending_data.csv',index=False)